In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/Othercomputers/My MacBook Pro/Projects/SR-INR

Mounted at /content/drive
/content/drive/Othercomputers/My MacBook Pro/Projects/SR-INR


## Train Dense INR
For image 1, 10, 15
* Iterations: 150000
* Layer size (lss): 50
* Number of layers: 15 (note these are all larger than the original COIN)
* Prune percentage: 40
* Refine iteration: 10000
* Change ld to results/image_{id}

For image 2, 7, 13
* Iterations: 50000
* Layer size(lss): 28
* Number of layers: 8 (original paper parameters)
* Prune percentage: 4
* Refine Iteration: 2000

For image 8, 17, 23
Updated training file (train.py), pruning/quantization/surp in separate file
* Iterations: 100000
* Layer size(lss): 36
* Number of layers: 12 (original paper parameters)



In [7]:
!python train.py --num_iters 100000 --layer_size 36 --num_layers 12 --image_id 8

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
Fitting SIREN to Kodak Image8...
Model size: 59.5kB
Full precision bpp: 1.21
100%|██████████████| 100000/100000 [38:10<00:00, 43.66it/s, best_psnr=21.1, loss=0.00815, psnr=20.9]
Best training psnr: 21.08


## Successive Refinement
* Takes a trained model as input
* Outputs a sparse model through the SuRP algorithm
* Model Directory: "./results/image_{iid}/best_model_{iid}.pt"

For large models (50 layer size, 15 layers):
* Total iterations: 100000
* Save image per 1000 iterations

For small models: (28 layer size, 8 layers):
* Total iterations: 50000
* Save image per 500 iterations.

In [ ]:
!python sr.py -it 100000 -iit 1000 -ld "./results/image_15/best_model_15.pt" -iid 15 --layer_size 50 --num_layers 15

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
/content/drive/Othercomputers/My MacBook Pro/Projects/SR-INR/surp.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly a

## Turn images into GIFs

In [ ]:
import os
import imageio

def create_gif_from_images(image_folder, output_gif, fps=5):
    """
    Create a GIF from a series of images in a folder.

    Args:
        image_folder (str): The path to the folder containing images.
        output_gif (str): The output path and filename for the GIF.
        fps (int): Frames per second (speed of the GIF).
    """
    # Get a sorted list of all images in the directory
    image_files = sorted([f for f in os.listdir(image_folder) if f.startswith('test') and f.endswith('.png')],
                         key=lambda x: int(x.replace('test', '').replace('.png', '')))

    # Check if there are images to process
    if not image_files:
        print("No images found in the directory!")
        return

    # Read and store images
    images = []
    for filename in image_files:
        filepath = os.path.join(image_folder, filename)
        images.append(imageio.imread(filepath))

    # Save images as a GIF
    imageio.mimsave(output_gif, images, fps=fps)
    print(f"GIF saved at {output_gif}")

# Path to the image folder and output GIF file (change appropriate folder names)
image_folder = "results/sr_images/image13"
output_gif_path = os.path.join(image_folder, "result_animation.gif")

# Create the GIF
create_gif_from_images(image_folder, output_gif_path, fps=10)


<ipython-input-8-405e04adbd79>:26: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filepath))


GIF saved at results/sr_images/image13/result_animation.gif


## New Model Compression File
Allows for running Mag_Pruning/Quantization/SuRP

### Quantization

In [3]:
!python compress.py --image_id 1 --compression_type Quantization --num_layers 15 --layer_size 50 --quant_level half

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
Traceback (most recent call last):
  File "/content/drive/Othercomputers/My MacBook Pro/Projects/SR-INR/compress.py", line 54, in <module>
    compressor = quantization(func_rep, args.image_id, args.compression_type, 
TypeError: quantization.__init__() takes 2 positional arguments but 7 were given
